In [1]:
!pip install -U -q tensorflow-addons

In [2]:
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

In [3]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft

In [4]:
path = ''

signal_type='raw'
input_type='raw'
n_channels=64
low_cut=20
high_cut=50
order=6 
window_size=50
overlap=50
fs=2048

whiten=True
center=False
extend=False
extend_size=0
normalize=False
mu=0

start_subject = 1
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
stochastic_depth_rate = 0.1

In [5]:
data, label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_size, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=0.2)

####################################################################################
Loading subject 1
####################################################################################
Whiten.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************


In [6]:
data.shape, label.shape, X_test.shape, y_test.shape

((17152, 64, 102, 1), (17152,), (4288, 64, 102), (4288,))

In [7]:
#Hyper-parameter

input_shape = data.shape[1:]
patch_size = 8
n_patches = 24 
dims = n_patches + 1 
n_transformer_layers = 2
n_heads = 2
transformer_units = [dims * 2,dims]  
mlp_head_units = [128, 34]  
n_classes = len(np.unique(label))
stochastic_depth = False

stochastic_depth_rate = 0.1

n_batches = 32
n_epochs = 150
n_folds = 3
type_of_experiment = 2

In [8]:
model = vit_gesture_classification(input_shape, dims, patch_size, n_patches,
                                  n_transformer_layers, n_heads, transformer_units,
                                   mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate)

model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 102, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 covolution_layer (Sequential)  (None, 32, 51, 25)   20413       ['input_1[0][0]']                
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 1600)  0           ['covolution_layer[0][0]']       
 tches)                                                                                           
                                                                                 

In [9]:
run_experiment(n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, extend, extend_size, center, whiten, ratio, noise_db, std,
                      input_shape, dims, patch_size, n_patches,
                       n_transformer_layers, n_heads, transformer_units,
                       mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate, type_of_experiment)

************************************************
Experiment Two
************************************************
####################################################################################
Loading subject 1
####################################################################################
Whiten.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
358/358 [==============================] - 26s 28ms/step - loss: 3.2417 - accuracy: 0.0929
Epoch 2/150
358/358 [==============================] - 10s 27ms/step - loss: 2.1328 - accuracy: 0.3249
Epoch 3/150
358/358 [==============================] - 11s 30ms

358/358 [==============================] - 9s 26ms/step - loss: 0.5411 - accuracy: 0.8226
Epoch 21/150
358/358 [==============================] - 10s 27ms/step - loss: 0.5316 - accuracy: 0.8229
Epoch 22/150
358/358 [==============================] - 9s 26ms/step - loss: 0.5055 - accuracy: 0.8343
Epoch 23/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4893 - accuracy: 0.8436
Epoch 24/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4826 - accuracy: 0.8457
Epoch 25/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4826 - accuracy: 0.8436
Epoch 26/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4518 - accuracy: 0.8520
Epoch 27/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4379 - accuracy: 0.8559
Epoch 28/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4408 - accuracy: 0.8611
Epoch 29/150
358/358 [==============================] - 9s 26ms/step - loss: 0.4201 -

360/360 [==============================] - 10s 28ms/step - loss: 0.2527 - accuracy: 0.9221
Epoch 92/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2553 - accuracy: 0.9220
Epoch 93/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2430 - accuracy: 0.9272
Epoch 94/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2576 - accuracy: 0.9194
Epoch 95/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2481 - accuracy: 0.9222
Epoch 96/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2374 - accuracy: 0.9253
Epoch 97/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2569 - accuracy: 0.9191
Epoch 98/150
360/360 [==============================] - 10s 28ms/step - loss: 0.2551 - accuracy: 0.9200
Score for fold  1: loss of 0.6464962959289551; accuracy of 81.26736283302307%
---------------------------------------------------
Training for fold 2 -------
Epoch 1/150
360/360 [==

360/360 [==============================] - 9s 25ms/step - loss: 0.3151 - accuracy: 0.9020
Epoch 65/150
360/360 [==============================] - 9s 24ms/step - loss: 0.3360 - accuracy: 0.8894
Epoch 66/150
360/360 [==============================] - 9s 24ms/step - loss: 0.3283 - accuracy: 0.8937
Epoch 67/150
360/360 [==============================] - 9s 24ms/step - loss: 0.3169 - accuracy: 0.9000
Epoch 68/150
360/360 [==============================] - 9s 26ms/step - loss: 0.3199 - accuracy: 0.8982
Score for fold  3: loss of 0.38356125354766846; accuracy of 87.75829076766968%
Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.6464962959289551 - Accuracy: 81.26736283302307%
-----------------------------------------------
> Fold 2 - Loss: 0.31479978561401367 - Accuracy: 90.97222089767456%
-----------------------------------------------
> Fold 3 - Loss: 0.38356125354766846 - Accuracy: 87.75829076766968%
-----------------------------------------------


352/352 [==============================] - 8s 23ms/step - loss: 0.2672 - accuracy: 0.9187
Epoch 61/150
352/352 [==============================] - 8s 22ms/step - loss: 0.2705 - accuracy: 0.9193
Epoch 62/150
352/352 [==============================] - 8s 22ms/step - loss: 0.2573 - accuracy: 0.9218
Epoch 63/150
352/352 [==============================] - 8s 23ms/step - loss: 0.2542 - accuracy: 0.9247
Epoch 64/150
352/352 [==============================] - 8s 22ms/step - loss: 0.2648 - accuracy: 0.9198
Epoch 65/150
352/352 [==============================] - 8s 22ms/step - loss: 0.2459 - accuracy: 0.9259
Epoch 66/150
352/352 [==============================] - 8s 23ms/step - loss: 0.2449 - accuracy: 0.9259
Epoch 67/150
352/352 [==============================] - 8s 23ms/step - loss: 0.2541 - accuracy: 0.9244
Epoch 68/150
352/352 [==============================] - 8s 23ms/step - loss: 0.2403 - accuracy: 0.9289
Epoch 69/150
352/352 [==============================] - 8s 23ms/step - loss: 0.2495 - 

...

Whiten.
Size of the input data is (17023, 64, 102, 1)
The input label shape is (17023,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
355/355 [==============================] - 13s 28ms/step - loss: 3.3386 - accuracy: 0.0624
Epoch 2/150
355/355 [==============================] - 10s 27ms/step - loss: 2.5440 - accuracy: 0.2073
Epoch 3/150
355/355 [==============================] - 10s 29ms/step - loss: 2.0462 - accuracy: 0.3213
Epoch 4/150
355/355 [==============================] - 10s 28ms/step - loss: 1.7495 - accuracy: 0.3928
Epoch 5/150
355/355 [==============================] - 10s 29ms/step - loss: 1.5697 - accuracy: 0.4570
Epoch 6/150
355/355 [==============================] - 

355/355 [==============================] - 10s 29ms/step - loss: 2.2686 - accuracy: 0.2758
Epoch 4/150
355/355 [==============================] - 11s 30ms/step - loss: 1.9301 - accuracy: 0.3604
Epoch 5/150
355/355 [==============================] - 11s 30ms/step - loss: 1.7038 - accuracy: 0.4293
Epoch 6/150
355/355 [==============================] - 11s 30ms/step - loss: 1.5225 - accuracy: 0.4817
Epoch 7/150
355/355 [==============================] - 11s 30ms/step - loss: 1.3779 - accuracy: 0.5342
Epoch 8/150
355/355 [==============================] - 11s 30ms/step - loss: 1.3128 - accuracy: 0.5497
Epoch 9/150
355/355 [==============================] - 11s 30ms/step - loss: 1.2228 - accuracy: 0.5796
Epoch 10/150
355/355 [==============================] - 11s 30ms/step - loss: 1.1542 - accuracy: 0.6054
Epoch 11/150
355/355 [==============================] - 11s 30ms/step - loss: 1.0994 - accuracy: 0.6248
Epoch 12/150
355/355 [==============================] - 11s 30ms/step - loss: 1.039

Epoch 82/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2787 - accuracy: 0.9132
Epoch 83/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2492 - accuracy: 0.9226
Epoch 84/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2399 - accuracy: 0.9253
Epoch 85/150
355/355 [==============================] - 10s 28ms/step - loss: 0.2386 - accuracy: 0.9247
Epoch 86/150
355/355 [==============================] - 10s 28ms/step - loss: 0.2405 - accuracy: 0.9255
Epoch 87/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2557 - accuracy: 0.9205
Epoch 88/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2540 - accuracy: 0.9211
Epoch 89/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2249 - accuracy: 0.9292
Epoch 90/150
355/355 [==============================] - 10s 29ms/step - loss: 0.2335 - accuracy: 0.9281
Epoch 91/150
355/355 [==============================] - 10s 29ms

Epoch 50/150
363/363 [==============================] - 10s 28ms/step - loss: 0.3579 - accuracy: 0.8867
Epoch 51/150
363/363 [==============================] - 11s 29ms/step - loss: 0.3562 - accuracy: 0.8887
Epoch 52/150
363/363 [==============================] - 11s 30ms/step - loss: 0.3381 - accuracy: 0.8939
Epoch 53/150
363/363 [==============================] - 11s 30ms/step - loss: 0.3194 - accuracy: 0.9001
Epoch 54/150
363/363 [==============================] - 11s 30ms/step - loss: 0.3388 - accuracy: 0.8957
Epoch 55/150
363/363 [==============================] - 11s 30ms/step - loss: 0.3396 - accuracy: 0.8910
Epoch 56/150
363/363 [==============================] - 11s 29ms/step - loss: 0.3185 - accuracy: 0.9008
Epoch 57/150
363/363 [==============================] - 11s 29ms/step - loss: 0.3174 - accuracy: 0.8995
Epoch 58/150
363/363 [==============================] - 11s 30ms/step - loss: 0.3304 - accuracy: 0.8976
Epoch 59/150
363/363 [==============================] - 11s 29ms

363/363 [==============================] - 10s 28ms/step - loss: 0.3851 - accuracy: 0.8821
Epoch 50/150
363/363 [==============================] - 10s 28ms/step - loss: 0.3794 - accuracy: 0.8804
Epoch 51/150
363/363 [==============================] - 10s 29ms/step - loss: 0.3788 - accuracy: 0.8820
Epoch 52/150
363/363 [==============================] - 10s 28ms/step - loss: 0.3600 - accuracy: 0.8860
Epoch 53/150
363/363 [==============================] - 10s 29ms/step - loss: 0.3559 - accuracy: 0.8879
Epoch 54/150
363/363 [==============================] - 10s 29ms/step - loss: 0.3460 - accuracy: 0.8905
Epoch 55/150
363/363 [==============================] - 10s 29ms/step - loss: 0.3530 - accuracy: 0.8906
Epoch 56/150
363/363 [==============================] - 10s 28ms/step - loss: 0.3390 - accuracy: 0.8928
Epoch 57/150
363/363 [==============================] - 10s 28ms/step - loss: 0.3497 - accuracy: 0.8937
Epoch 58/150
363/363 [==============================] - 10s 28ms/step - loss: